In [6]:
import tensorflow as tf
import numpy as np
import json
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Carica il modello TFLite
def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

# Carica i dati MFCC dal file JSON
def load_mfcc_and_labels_from_json(json_path):
    with open(json_path, "r") as file:
        data = json.load(file)

    if "mfcc" in data:
        mfcc_array = np.array(data["mfcc"], dtype=np.float32)
        labels = data["labels"]  # Assumiamo che ci siano le etichette nel campo "labels"
    else:
        mfcc_array = np.array(data, dtype=np.float32)
        labels = []  # In caso di struttura diversa

    print("Forma dell'MFCC array:", mfcc_array.shape)
    print("Numero di etichette:", len(labels))

    # Controlla se la forma dell'array MFCC è quella corretta
    if len(mfcc_array.shape) == 2:  # se MFCC è una matrice 2D di forma [35, 13]
        mfcc_array = np.expand_dims(mfcc_array, axis=0)  # Aggiungi una dimensione per il batch
        print("Nuova forma dell'MFCC array:", mfcc_array.shape)

    return mfcc_array, labels


# Controlla se la forma dell'array MFCC è quella corretta


instrument_names = [
         "Keyboard",
        "Guitar",
        "Flute",
        "Vocal",
        "Organ",
        "Reed ",
        "Mallet",
        "String",
        "Brass ",
        "Bass"
]

# Funzione per fare la predizione
def predict_instrument(interpreter, mfcc_array):
    # Ottieni dettagli su input e output del modello
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Controlla la forma dell'input richiesta
    input_shape = input_details[0]['shape']
    print(f"Forma attesa del modello TFLite: {input_shape}")

    predictions = []
    for instance in mfcc_array:
        input_data = instance.reshape(input_shape)  # Modifica la forma dell'input
        interpreter.set_tensor(input_details[0]['index'], input_data)  # Imposta il dato di input
        interpreter.invoke()  # Invia il dato al modello
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predicted_class = np.argmax(output_data)
        instrument_name = instrument_names[predicted_class]
        predictions.append(instrument_name)

    return predictions  # Restituisce le predizioni

# Percorsi dei file
tflite_model_path = "drive/MyDrive/Colab Notebooks/model.tflite"
mfcc_json_path = "drive/MyDrive/Colab Notebooks/dataset/mfccfin.json"

# Carica il modello e l'MFCC
interpreter = load_tflite_model(tflite_model_path)
mfcc_array, true_labels = load_mfcc_and_labels_from_json(mfcc_json_path)
true_labels_instrument_names = [instrument_names[label] for label in true_labels]

# Effettua la predizione
predicted_instruments = predict_instrument(interpreter, mfcc_array)

# Percentuale di accuratezza
correct_predictions = 0
total_predictions = len(true_labels_instrument_names)

for true_label, predicted_instrument in zip(true_labels_instrument_names, predicted_instruments):
    if true_label == predicted_instrument:
        correct_predictions += 1

accuracy = (correct_predictions / total_predictions) * 100

# Stampa la percentuale di accuratezza
print(f"Percentuale di accuratezza: {accuracy:.2f}%")

# Stampa il risultato
for idx, (true_label, predicted_instrument) in enumerate(zip(true_labels_instrument_names, predicted_instruments), start=1):
    print(f"Lo strumento numero {idx} è: Etichetta vera: {true_label}, Predizione: {predicted_instrument}" )



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Forma dell'MFCC array: (1965, 35, 13)
Numero di etichette: 1965
Forma attesa del modello TFLite: [ 1 35 13]
Percentuale di accuratezza: 91.45%
Lo strumento numero 1 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 2 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 3 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 4 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 5 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 6 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 7 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 8 è: Etichetta vera: Keyboard, Predizione: Guitar
Lo strumento numero 9 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strumento numero 10 è: Etichetta vera: Keyboard, Predizione: Keyboard
Lo strume